In [1]:
!git clone https://github.com/facebookresearch/segment-anything-2.git
%cd /kaggle/working/segment-anything-2
%pip install -e .
%cd /kaggle/working/segment-anything-2/checkpoints/
!bash /kaggle/working/segment-anything-2/checkpoints/download_ckpts.sh
%cd /kaggle/working/segment-anything-2/

Cloning into 'segment-anything-2'...
remote: Enumerating objects: 1070, done.
remote: Total 1070 (delta 0), reused 0 (delta 0), pack-reused 1070 (from 1)
Receiving objects: 100% (1070/1070), 134.70 MiB | 16.88 MiB/s, done.
Resolving deltas: 100% (376/376), done.
/kaggle/working/segment-anything-2
Obtaining file:///kaggle/working/segment-anything-2
  Installing build dependencies ... done
  Checking if build backend supports build_editable ... done
  Getting requirements to build editable ... done
  Preparing editable metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.2/42.2 kB 1.9 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 154.5/154.5 kB 8.3 MB/s eta 0:00:00
  Building editable for SAM-2 (pyproject.toml) ... done
  Created wheel for SAM-2: filename=SAM_2-1.0-0.editable-cp310-cp310-linux_x86_64.whl size=13818 sha256=e0045ce155d41b79228306809bd8a432af8c40c5f88a71fe627c493f2c5946fe
  Stored in dire

In [2]:
!wget https://storage.yandexcloud.net/ds-ods/files/files/2ad601fd/test.zip
!pip install supervision 

--2025-01-27 20:56:35--  https://storage.yandexcloud.net/ds-ods/files/files/2ad601fd/test.zip
Resolving storage.yandexcloud.net (storage.yandexcloud.net)... 213.180.193.243, 2a02:6b8::1d9
Connecting to storage.yandexcloud.net (storage.yandexcloud.net)|213.180.193.243|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1444862118 (1.3G) [application/x-zip-compressed]
Saving to: ‘test.zip’

test.zip            100%[===================>]   1.34G  15.1MB/s    in 95s     

2025-01-27 20:58:11 (14.5 MB/s) - ‘test.zip’ saved [1444862118/1444862118]

--2025-01-27 20:58:11--  https://storage.yandexcloud.net/ds-ods/files/submissions/21dd71fc-63e7-4419-8cc6-ec6af1214dee/62ab2c57/NTO_SUB0%202.zip
Resolving storage.yandexcloud.net (storage.yandexcloud.net)... 213.180.193.243, 2a02:6b8::1d9
Connecting to storage.yandexcloud.net (storage.yandexcloud.net)|213.180.193.243|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 81104172 (77M) [application/zip]
Sav

In [3]:
import os
# if using Apple MPS, fall back to CPU for unsupported ops
os.environ["PYTORCH_ENABLE_MPS_FALLBACK"] = "1"
import numpy as np
import torch
import matplotlib.pyplot as plt
from PIL import Image

In [4]:
%%capture
!unzip /kaggle/working/segment-anything-2/test.zip

In [5]:
# select the device for computation
if torch.cuda.is_available():
    device = torch.device("cuda")
elif torch.backends.mps.is_available():
    device = torch.device("mps")
else:
    device = torch.device("cpu")
print(f"using device: {device}")

if device.type == "cuda":
    # use bfloat16 for the entire notebook
    torch.autocast("cuda", dtype=torch.bfloat16).__enter__()
    # turn on tfloat32 for Ampere GPUs (https://pytorch.org/docs/stable/notes/cuda.html#tensorfloat-32-tf32-on-ampere-devices)
    if torch.cuda.get_device_properties(0).major >= 8:
        torch.backends.cuda.matmul.allow_tf32 = True
        torch.backends.cudnn.allow_tf32 = True
elif device.type == "mps":
    print(
        "\nSupport for MPS devices is preliminary. SAM 2 is trained with CUDA and might "
        "give numerically different outputs and sometimes degraded performance on MPS. "
        "See e.g. https://github.com/pytorch/pytorch/issues/84936 for a discussion."
    )

using device: cuda


In [9]:
from sam2.build_sam import build_sam2
from sam2.sam2_image_predictor import SAM2ImagePredictor

sam2_checkpoint = "./sm_base_3ep.pth"
model_cfg = "configs/sam2.1/sam2.1_hiera_b+.yaml"

sam2_model_base = build_sam2(model_cfg, sam2_checkpoint, device=device)

predictor_base = SAM2ImagePredictor(sam2_model_base)

In [10]:
%%capture
!pip install ultralytics supervision

In [11]:
from  ultralytics import YOLO
import supervision as sv
model = YOLO('/kaggle/input/best-yolo/best (1).pt')

def predict_yolo_image(image):
    return sv.Detections.from_ultralytics(model(image,conf=0.4)[0])

Creating new Ultralytics Settings v0.0.6 file ✅ 
View Ultralytics Settings with 'yolo settings' or at '/root/.config/Ultralytics/settings.json'
Update Settings with 'yolo settings key=value', i.e. 'yolo settings runs_dir=path/to/dir'. For help see https://docs.ultralytics.com/quickstart/#ultralytics-settings.


In [12]:
def predict_sam(image,box,predictor):
    predictor.set_image(image)
    masks, scores, _ = predictor.predict(
        point_coords=None,
        point_labels=None,
        box=box,
        return_logits=True,
        multimask_output=False,
    )
    return masks, scores

In [13]:
np.random.seed(3)

def show_mask(mask, ax, random_color=False, borders = True):
    if random_color:
        color = np.concatenate([np.random.random(3), np.array([0.6])], axis=0)
    else:
        color = np.array([30/255, 144/255, 255/255, 0.6])
    h, w = mask.shape[-2:]
    mask = mask.astype(np.uint8)
    mask_image =  mask.reshape(h, w, 1) * color.reshape(1, 1, -1)
    if borders:
        import cv2
        contours, _ = cv2.findContours(mask,cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_NONE) 
        # Try to smooth contours
        contours = [cv2.approxPolyDP(contour, epsilon=0.01, closed=True) for contour in contours]
        mask_image = cv2.drawContours(mask_image, contours, -1, (1, 1, 1, 0.5), thickness=2) 
    ax.imshow(mask_image)

def show_points(coords, labels, ax, marker_size=375):
    pos_points = coords[labels==1]
    neg_points = coords[labels==0]
    ax.scatter(pos_points[:, 0], pos_points[:, 1], color='green', marker='*', s=marker_size, edgecolor='white', linewidth=1.25)
    ax.scatter(neg_points[:, 0], neg_points[:, 1], color='red', marker='*', s=marker_size, edgecolor='white', linewidth=1.25)   

def show_box(box, ax):
    x0, y0 = box[0], box[1]
    w, h = box[2] - box[0], box[3] - box[1]
    ax.add_patch(plt.Rectangle((x0, y0), w, h, edgecolor='green', facecolor=(0, 0, 0, 0), lw=2))    

def show_masks(image, masks, scores, point_coords=None, box_coords=None, input_labels=None, borders=True):
    for i, (mask, score) in enumerate(zip(masks, scores)):
        plt.figure(figsize=(10, 10))
        plt.imshow(image)
        show_mask(mask, plt.gca(), borders=borders)
        if point_coords is not None:
            assert input_labels is not None
            show_points(point_coords, input_labels, plt.gca())
        if box_coords is not None:
            # boxes
            show_box(box_coords, plt.gca())
        if len(scores) > 1:
            plt.title(f"Mask {i+1}, Score: {score:.3f}", fontsize=18)
        plt.axis('off')
        plt.show()

In [14]:
from tqdm.auto import tqdm
import supervision as sv
import pycocotools
import json
import pycocotools.mask
import numpy as np
from ultralytics import YOLO
import albumentations as A
import os

test_folder = "test"
output_folder = "output_yolo"  # Папка для сохранения результатов
os.makedirs(output_folder, exist_ok=True)
predictions = []
for file_name in tqdm(os.listdir(test_folder)):
    image_path = os.path.join(test_folder, file_name)
    image = Image.open(image_path)
    predicts = predict_yolo_image(image)

    if predicts.xyxy is None:
        continue
    pred_classes = predicts.class_id.tolist()  # Классы объектов
    pred_boxes = predicts.xyxy.tolist()#instances.pred_boxes.tensor.tolist()  # Координаты bbox
    pred_scores = predicts.confidence.tolist()  # Скоринг предсказаний
        
    for i in range(len(pred_boxes)):
        mask , scores = predict_sam(image,pred_boxes[i],predictor_base)
        if float(scores[0]) <= 0.5:
            continue
        rle = pycocotools.mask.encode(np.asfortranarray(mask.astype(np.uint8)[0]))
        predictions.append({
            "image_name": file_name,
            "category_id": 0,  # ID категории
            "bbox": pred_boxes[i],  # Координаты bounding box
            "score": float(scores[0] * 0.5 + pred_scores[i] * 0.5),  # Оценка уверенности
            "segmentation": {
                "size" : rle["size"],
                "counts" : str(rle["counts"])
            }  # Сегментация в формате RLE
        })


# Сохранение предсказаний в JSON
predictions_file = os.path.join(output_folder, "submission.json")
with open(predictions_file, "w") as f:
    json.dump(predictions, f, indent=4)

print(f"Обработка завершена. Результаты сохранены в папке: {output_folder}")

  0%|          | 0/9074 [00:00<?, ?it/s]


0: 544x1024 1 Animals, 112.8ms
Speed: 16.7ms preprocess, 112.8ms inference, 215.0ms postprocess per image at shape (1, 3, 544, 1024)

0: 544x1024 1 Animals, 110.3ms
Speed: 3.8ms preprocess, 110.3ms inference, 2.3ms postprocess per image at shape (1, 3, 544, 1024)

0: 544x1024 1 Animals, 110.1ms
Speed: 3.4ms preprocess, 110.1ms inference, 2.2ms postprocess per image at shape (1, 3, 544, 1024)

0: 544x1024 1 Animals, 110.2ms
Speed: 3.7ms preprocess, 110.2ms inference, 2.4ms postprocess per image at shape (1, 3, 544, 1024)

0: 544x1024 1 Animals, 110.4ms
Speed: 3.4ms preprocess, 110.4ms inference, 2.3ms postprocess per image at shape (1, 3, 544, 1024)

0: 544x1024 1 Animals, 110.2ms
Speed: 3.6ms preprocess, 110.2ms inference, 2.1ms postprocess per image at shape (1, 3, 544, 1024)

0: 544x1024 1 Animals, 110.4ms
Speed: 3.6ms preprocess, 110.4ms inference, 2.4ms postprocess per image at shape (1, 3, 544, 1024)

0: 544x1024 1 Animals, 110.3ms
Speed: 3.4ms preprocess, 110.3ms inference, 2.2m